## Hugging face 라이브러리: Natural Language Processing 8 Task
huggng face : https://huggingface.co/docs/transformers/v4.36.1/en/index  
leader board : https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard  
[실습] Task 별로 좋은 성능을 보이는 한국어 model 찾아서 사용해 보기

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 사용할 폴더로 이동
%cd /content/gdrive/MyDrive/빅데이터/12. 자연어 처리/day4

/content/gdrive/MyDrive/빅데이터/12. 자연어 처리/day4


In [ ]:
from transformers import pipeline

#### Task1. 감성분석 (Text classification)  
##### : 입력된 텍스트에 있는 긍/부 or 추가 감성을 분석함

In [ ]:
# basemodel: "sentiment-analysis"
Classifier = pipeline("sentiment-analysis")
Classifier("I am honored to be able to give this lecture.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9996919631958008}]

In [ ]:
# 복수개의 감성분석, 결과 만들기
results = Classifier(["I am honored to be able to give this lecture.", "I'm so tired today"])
for result in results:
  print(f"label : {result['label']}, with score : {round(result['score'], 4)}")

label : POSITIVE, with score : 0.9997
label : NEGATIVE, with score : 0.9998


In [ ]:
# Hugging face에서 찾은 한국어 감성분석 모델 사용해보기 1
# "WhitePeak/bert-base-cased-Korean-sentiment"
sentiment_model = pipeline(model = "WhitePeak/bert-base-cased-Korean-sentiment")
sentiment_model("넌 너무 편리한 것 같아.")

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9385913014411926}]

In [ ]:
# Hugging face에서 찾은 한국어 감성분석 모델 사용해보기 2
# "snunlp/KR-FinBert-SC"
FinBert_model = pipeline(model = "snunlp/KR-FinBert-SC")
FinBert_model("이렇게 해서 쓰면 저작권문제로 번질 수 있다")

[{'label': 'negative', 'score': 0.7903022766113281}]

#### Task2. 질의 응답 (Question Answering)
##### : 주어진 context에 기반해 질문에 대한 응답 (MRC: Machine Reading Comprehension)

In [ ]:
# basemodel: "question-answering"
question_answer = pipeline("question-answering")
question_answer(
    question = "What is your name?",
    context = "My name is Joshua and I'm learning at Chunjae in Seoul"
)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9982267022132874, 'start': 11, 'end': 17, 'answer': 'Joshua'}

In [ ]:
# Hugging face에서 찾은 한국어 질의응답 모델 사용해보기 1
# "timpal0l/mdeberta-v3-base-squad2"
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")
question = "넌 뭐하고 있어?"
context = "나는 소영이고 서울에 살아. 천재교육에서 교육을 듣고 있어"
qa_model(question=question, context=context)

{'score': 0.007958896458148956, 'start': 15, 'end': 22, 'answer': ' 천재교육에서'}

In [ ]:
# Hugging face에서 찾은 한국어 질의응답 모델 사용해보기 2
# 'uomnf97/klue-roberta-finetuned-korquad-v2'

# 직접 tokenizer/ model을 활용할 수 있음
# from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer
# config = AutoConfig.from_pretrained('uomnf97/klue-roberta-finetuned-korquad-v2')
# tokenizer = AutoTokenizer.from_pretrained('uomnf97/klue-roberta-finetuned-korquad-v2')
# model = AutoModelForQuestionAnswering.from_pretrained('uomnf97/klue-roberta-finetuned-korquad-v2',config=config)

#### Task3. 제로샷 분류(Zero-Shot Classification)
##### : 학습과 예제 제공없이, 텍스트와 클래스 라벨을 제공하고 가장 유사한 라벨로 분류


In [ ]:
# basemodel: "zero-shot-classification"
zeroshot_model = pipeline("zero-shot-classification")
sequence_to_classfity = "One day I will see the world"
candidate_labels = ["travel", "cooking", "dancing"]
zeroshot_model(
    sequence_to_classfity,
    candidate_labels=candidate_labels
)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'sequence': 'One day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9941015839576721, 0.0031261490657925606, 0.002772233448922634]}

In [ ]:
# [실습] Hugging face에서 찾은 한국어 제로샷 모델 사용해보기 1
zeroshot_ko_model = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
sequence_to_classfity = "나는 바다를 보러 갈거야"
candidate_labels = ["여행", "요리", "노래하기"]
zeroshot_ko_model(
    sequence_to_classfity,
    candidate_labels=candidate_labels
)

{'sequence': '나는 바다를 보러 갈거야',
 'labels': ['여행', '노래하기', '요리'],
 'scores': [0.9917282462120056, 0.004486265126615763, 0.0037854767870157957]}

#### Task4. 요약(Summarization)
##### : 입력된 텍스트를 요약하여 제공


In [ ]:
text = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [ ]:
# basemodel: "summarization"
summarizer = pipeline("summarization")
summarizer(text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, nine of them between 1999 and 2002 . She is believed to still be married to four men, and at one time, she was married to eight men at once .'}]

In [ ]:
ko_text = "2차전지 생산기지 : 중국 중심 속 미국과 독일 간 경쟁 구도전세계 2차전지 3대 생산기지는 중국, 미국, 독일로 전세계 2차전지 생산의 약 80%를 차지하고 있으며, 이중 중국이 대부분을 차지했다. S&P Global MarketIntelligence에 따르면, 2023년 5월 기준 전세계 2차전지 생산 규모는 2.8TWh(Tera WattHour)이며, 중국이 전세계 생산의 약 73%인 2,052GWh(Giga Watt Hour)로 1위 생산국 입지를 보였다. S&P는 향후 2030년에도 전세계 2차전지 생산의 약 57%로 여전히 1위 입지가 유지될 것으로 전망된다. 美 인플레이션 감축법(IRA)과 EU의 배터리법(BatteryRegulation) 등 영향으로 중국의 생산 비중이 2023년보다는 축소될 것이나 여전히세계 최대 생산국 지위를 유지할 전망이다.한편, 2차전지 제조 기업의 경우, 전세계 전기차용 2차전지 판매 비중은 중국기업이 전체의 50%를 상회하고 있다. SNE 리서치에 따르면, 2023(1~8월) 전기차용 2차전지 판매 기준 세계 Top 10 내 중국기업 비중은 63%로 한국기업과 일본기업 비중을압도하고 있다. 특히, CATL과 BYD 등 2개의 중국기업 비중만 50%를 넘는 비중을 차지하고 있다. 2차전지 핵심광물 주도권 : 중국 편중 심화리튬, 흑연 등 2차전지를 구성하는 주요 소재 제조에 사용되는 주요 광물에 대한 제련 의존도가 중국에 편중되어 있는 것으로 나타났다. 미국지질조사국(USGS)에 따르면,2차전지의 양극재(Cathode Materials)에 쓰이는 리튬(호주 46.9%), 니켈(인도네시아48.5%), 코발트(콩고 68.4%) 등 국가에서의 전세계 생산 비중이 가장 높다. 또, 음극재(Anode Materials)로 사용되는 흑연은 전세계의 65.4%가 중국에서 생산될 뿐 아니라매장량도 15.8%로 튀르키예 27.3% 다음으로 높은 비중을 차지하고 있다. 다만, 핵심광물들의 제련 및 셀(Cell) 가공 의존도는 70% 이상이 중국에 편중되 있어 실질적인 2차전지 광물 공급망 주도권은 중국이 장악한 것으로 분석된다. 아울러 2차전지용으로 사용되는 인조흑연 등을 포함한 총 12개 주요 원자재(탄산리튬 및 황산니켈 등 제외) 대한 전세계 공급망도 하고 대부분 중국에 의존하는 것으로 분석됐다. 특히, 천연흑연,산화 및 수산화리튬, 산화 및 수산화니켈 등 5개 항목은 전세계 수출의 50% 이상을중국이 차지했다."

In [ ]:
# [실습] Hugging face에서 찾은 한국어 요약 모델 사용해보기 1
ko_summarizer = pipeline("summarization", model = "noahkim/KoT5_news_summarization")
ko_summarizer(ko_text, max_length=200)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'summary_text': 'S&P는 향후 2030년에도 전세계 2차전지 생산의 약 57%로 여전히 1위 입지가 유지될 것으로 전망되며 2차전지 핵심광물 주도권 : 중국 편중 심화리튬, 흑연 등 2차전지를 구성하는 주요 소재 제조에 사용되는 주요 광물에 대한 제련 의존도가 중국에 편중되어 있어 실질적인 2차전지 광물 공급망 주도권은 중국이 장악한 것으로 분석된다.'}]

#### Task5. 텍스트 생성(Text Generation)
##### : 프롬프트를 모델에 제공하면 모델이 나머지 텍스트를 생성하여 문장을 완성함 (자동완성 기능)

In [ ]:
# basemodel: 'text-generation'
generator = pipeline('text-generation')
generator("In this course, we will teach you how to NLP",
          max_length=30,
          num_return_sequences=2)

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to NLP and our knowledge of their properties, which can help you become aware of them. But it'},
 {'generated_text': 'In this course, we will teach you how to NLP and the many popular NLP applications.\n\nBefore we begin, you do not need'}]

In [ ]:
# [실습] Hugging face에서 찾은 한국어 텍스트 생성 모델 사용해보기 1
ko_generator = pipeline('text-generation', model="CohereForAI/c4ai-command-r-plus")
ko_generator("나는 여기서 코딩을 배우며 자연어 처리와 관련된 프로젝트를",
          max_length=100,
          num_return_sequences=2)
# 너무 오래걸려서 스탑...^^

model-00006-of-00044.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

model-00007-of-00044.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

model-00008-of-00044.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

model-00009-of-00044.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

model-00010-of-00044.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

model-00011-of-00044.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

model-00012-of-00044.safetensors:   0%|          | 0.00/4.63G [00:00<?, ?B/s]

model-00013-of-00044.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

KeyboardInterrupt: 

#### Task6. 마스크 채우기(Fill-mask)
##### : 토큰에 들어 갈 단어를 예측하여 가장 가능성 있는 단어를 받기

In [ ]:
# basemodel: "fill-mask"
nmasker = pipeline("fill-mask")
nmasker("I'm going to hospital and meet a <mask>", top_k=2)

No model was supplied, defaulted to distilbert/distilroberta-base and revision ec58a5b (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.19275860488414764,
  'token': 3299,
  'token_str': ' doctor',
  'sequence': "I'm going to hospital and meet a doctor"},
 {'score': 0.0679468959569931,
  'token': 27321,
  'token_str': ' psychiatrist',
  'sequence': "I'm going to hospital and meet a psychiatrist"}]

In [ ]:
# [실습] Hugging face에서 찾은 한국어 마스크 채우기 모델 사용해보기 1
unmasker = pipeline('fill-mask', model='xlm-roberta-base')
unmasker("안녕 나는 <mask>에서 살고 있는 솔이라고 해", top_k=3)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

[{'score': 0.11341188102960587,
  'token': 20899,
  'token_str': '서울',
  'sequence': '안녕 나는 서울 에서 살고 있는 솔이라고 해'},
 {'score': 0.09730132669210434,
  'token': 9397,
  'token_str': '한국',
  'sequence': '안녕 나는 한국 에서 살고 있는 솔이라고 해'},
 {'score': 0.07706746459007263,
  'token': 20598,
  'token_str': '미국',
  'sequence': '안녕 나는 미국 에서 살고 있는 솔이라고 해'}]

#### Task7. 개체명 인식 (NER)

In [ ]:
# basemodel: "ner"
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging face in Brooklyn")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-p

[{'entity_group': 'PER',
  'score': 0.9986724,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.940561,
  'word': 'Hugging face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.97993374,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [ ]:
# [실습] Hugging face에서 찾은 한국어 개체명 모델 사용해보기 1
ko_ner = pipeline("ner", model = "xlm-roberta-large-finetuned-conll03-english", grouped_entities=True)
ko_ner("내 이름은 하영이고 서울에서 일해")

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9998486,
  'word': '하영',
  'start': 6,
  'end': 8},
 {'entity_group': 'LOC',
  'score': 0.99999416,
  'word': '서울',
  'start': 11,
  'end': 13}]

#### Task8. 번역 (Translation)

In [ ]:
# model: "Helsinki-NLP/opus-mt-ko-en"
translator = pipeline("translation", "Helsinki-NLP/opus-mt-ko-en")
translator("한국말을 영어로 번역해주세요. 안녕!")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Please translate Korean into English. Bye!'}]

In [ ]:
# [실습] Hugging face에서 찾은 한국어 번역 모델 사용해보기 1 (한국 ->영어 / 영어 -> 한국)
ko_translator = pipeline("translation", model = "facebook/mbart-large-50-many-to-many-mmt")
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."
ko_translator.src_lang = "hi_IN"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"]
)
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# # => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

# # translate Arabic to English
# tokenizer.src_lang = "ar_AR"
# encoded_ar = tokenizer(article_ar, return_tensors="pt")
# generated_tokens = model.generate(
#     **encoded_ar,
#     forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
# )
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# # => "The Secretary-General of the United Nations says there is no military solution in Syria."


In [ ]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['UN Secretary General이 말하길, "Syria에는 군사적 해결책이 없습니다."']